<a href="https://colab.research.google.com/github/AlyGamalAbusenna/Multiclass-Classification-with-Pins-Face-Dataset/blob/main/crop_faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import os
import cv2
import shutil
from PIL import Image
import numpy as np

class FaceExtractor:
    def __init__(self, prototxt_path='deploy.prototxt', model_path='res10_300x300_ssd_iter_140000.caffemodel'):
        # Load pre-trained face detection model from Caffe
        self.face_net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

    def extract_faces_from_zip(self, input_zip_path, output_zip_path):
        # Create a temporary directory to store the extracted zip content
        with zipfile.ZipFile(input_zip_path, 'r') as zip_ref:
            temp_dir = 'temp_extracted'
            zip_ref.extractall(temp_dir)

        cropped_faces_dir = 'cropped_faces'
        if os.path.exists(cropped_faces_dir):
            shutil.rmtree(cropped_faces_dir)
        os.makedirs(cropped_faces_dir, exist_ok=True)

        for foldername, subfolders, filenames in os.walk(temp_dir):
            for filename in filenames:
                if filename.endswith(('.png', '.jpg', '.jpeg')):
                    file_path = os.path.join(foldername, filename)
                    image = cv2.imread(file_path)
                    if image is None:
                        continue

                    (h, w) = image.shape[:2]
                    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
                    self.face_net.setInput(blob)
                    detections = self.face_net.forward()

                    for i in range(0, detections.shape[2]):
                        confidence = detections[0, 0, i, 2]
                        if confidence > 0.5:  # Minimum confidence threshold
                            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                            (startX, startY, endX, endY) = box.astype("int")

                            # Ensure coordinates are within image bounds
                            startX = max(0, startX)
                            startY = max(0, startY)
                            endX = min(w, endX)
                            endY = min(h, endY)

                            face = image[startY:endY, startX:endX]
                            if face.size == 0:
                                continue

                            # Create directory structure in output if not already present
                            relative_path = os.path.relpath(foldername, temp_dir)
                            target_folder = os.path.join(cropped_faces_dir, relative_path)
                            os.makedirs(target_folder, exist_ok=True)

                            # Save cropped face
                            cropped_face_path = os.path.join(target_folder, f"face_{filename}")
                            cv2.imwrite(cropped_face_path, face)

        # Zip the cropped faces directory
        shutil.make_archive(output_zip_path.replace('.zip', ''), 'zip', cropped_faces_dir)

        # Cleanup temporary directories
        shutil.rmtree(temp_dir)
        shutil.rmtree(cropped_faces_dir)


input_zip_path = 'archive.zip'
output_zip_path = 'cropped_faces.zip'
face_extractor = FaceExtractor()
face_extractor.extract_faces_from_zip(input_zip_path, output_zip_path)
print("The cropped faces have been saved in the new zip file.")


The cropped faces have been saved in the new zip file.
